In [15]:
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import math
import os
import csv

In [ ]:
def threshold(img):
    result = np.zeros(img.shape)
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            if img[y,x]>128: result[y,x]=1
    return result

In [16]:
labels = ['celana_panjang', 'celana_pendek', 'Kaos']
imgPerLabels = 200
orientation = [0, 45, 90, 135]
datasets = {}
input_folder = 'dataset_closed/'

for j in range(len(labels)):
    datasets[labels[j]] = np.empty((imgPerLabels, 256, 256))
    count=0
        # if os.path.isdir(labels[j]+"\\"+str(imgindex)):
    for filename in os.listdir(input_folder):
        if(labels[j] in filename):
            img_path = os.path.join(input_folder, filename)
            img = cv.cvtColor(cv.resize(cv.imread(img_path), (256, 256)), cv.COLOR_BGR2GRAY)
            datasets[labels[j]][count] = img
            count+=1
            datasets[labels[j]][count] = cv.rotate(img, cv.ROTATE_180)
            count+=1
        if count==imgPerLabels: break

In [17]:
def getGlcm(datasets:np.ndarray, ori:np.ndarray[int, int], dist=1):
    result = {}
    
    for i in datasets:
        print(i)
        label = []
        for j in range(len(datasets[i])):
            img:np.ndarray = datasets[i][j]
            imgGlcm = {}
            for k in range(len(ori)):
                (iH, iW) = img.shape[:2]
                matriks = np.zeros((256, 256), float)
                startX = startY = 0
                endX = iW
                endY = iH
                
                if ori[k]==0: endX=iW-dist
                elif ori[k]==90: startY=dist
                elif ori[k]==45:
                    startY=dist
                    endX=iW-dist
                elif ori[k]==135:
                    startY=startX=dist
                
                for y in range(startY, endY):
                    for x in range(startX, endX):
                        if ori[k]==0: matriks[int(img[y,x]), int(img[y,x+dist])] =+ 1
                        elif ori[k]==90: matriks[int(img[y,x]), int(img[y-dist,x])] =+ 1
                        elif ori[k]==45: matriks[int(img[y,x]), int(img[y-dist,x+dist])] =+ 1
                        elif ori[k]==135: matriks[int(img[y,x]), int(img[y-dist,x-dist])] =+ 1

                matriks +=matriks.transpose()
                summary = np.sum(matriks)
                matriks /= summary
                # for y in range(matriks.shape[0]):
                #     for x in range(matriks.shape[1]):
                #         if matriks[y,x] != 0: matriks[y,x] /= summary
                
                imgGlcm[orientation[k]] = matriks
            label.append(imgGlcm)
        result[i] = label
        print(i)
    return result

glcm = getGlcm(datasets, orientation)

celana_panjang
celana_panjang
celana_pendek
celana_pendek
Kaos
Kaos


In [18]:
def getFeature(glcm:dict[int, dict[int, np.ndarray]], features:list[str]=['contrast', 'dissimiliarity', 'homogeneity', 'entropy', 'asm', 'corellation', 'energy']):
    
    result = []
    for i in glcm.keys():
        for j in range(len(glcm[i])):
            imgFeatures = {}
            for k in glcm[i][j].keys():
                matriks = glcm[i][j][k]
                
                fitur = {f'contrast{k}':0.0, 
                            f'dissimiliarity{k}':0.0, 
                            f'homogeneity{k}':0.0, 
                            f'entropy{k}':0.0, 
                            f'asm{k}':0.0, 
                            f'corellation{k}':0.0, 
                            f'energy{k}':0.0}
                
                meanY = np.mean(matriks)
                meanX = np.mean(matriks.transpose())
                
                for y in range(256):
                    for x in range(256):
                        if matriks[y,x]==0.0: continue
                        if "contrast" in features: fitur[f"contrast{k}"] += matriks[y,x]*math.pow(y-x, 2)
                        if "dissimiliarity" in features: fitur[f"dissimiliarity{k}"] += matriks[y,x]*abs(y-x)
                        if "homogeneity" in features: fitur[f"homogeneity{k}"] += matriks[y,x]/(1+math.pow(y-x, 2))
                        if "entropy" in features: fitur[f"entropy{k}"] -= matriks[y,x]*math.log10(matriks[y,x])
                        if "asm" in features: fitur[f"asm{k}"] += math.pow(matriks[y,x], 2)
                
                if "corellation" in features:
                    stdX = np.std(matriks)
                    stdY = np.std(matriks.transpose())
                    if stdX != 0 and stdY != 0: fitur[f"corellation{k}"] = (- (meanX * meanY)) / (stdX * stdY)
                if 'energy' in features: fitur[f'energy{k}'] = math.sqrt(fitur[f"asm{k}"])
                
                for l in fitur:
                    imgFeatures[l] = fitur[l]
            
            imgFeatures['label'] = i        
            result.append(imgFeatures)
    return result
features = getFeature(glcm)

In [19]:
header = []

for head in features[0].keys():
    header.append(head)

with open('Features.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(features)